In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
import numpy as np
from statistics import mean
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from technical_indicators import calculate_daily_relative_difference

In [17]:
def retrieve_data(df, filename, varname):
    df_new = pd.read_csv(filename)
    df_new["Date"] = pd.to_datetime(df_new["Date"])
    try:
        df_new.columns = ["Date", "Close", "Open", "High", "Low", "Vol.", "Change%"]
    except ValueError:
        try:
            df_new.columns = ["Date", "Close", "Open", "High", "Low", "Change%"]
        except ValueError:
            df_new.columns = ["Date", "Close", "Open", "High", "Low"]

    rows = []
    for index, row in df_new.iterrows():
        date = row['Date']
        p_close = float(str(row['Close']).replace(',', ''))
        p_open = float(str(row['Open']).replace(',', ''))
        p_high = float(str(row['High']).replace(',', ''))
        p_low = float(str(row['Low']).replace(',', ''))
        if df_new.columns.tolist() == ["Date", "Close", "Open", "High", "Low", "Vol.", "Change%"]:
            if str(row['Vol.']) == "-":
                volume = 0
            else:
                if row['Vol.'][-1] == "B": # volume in billions
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000000000
                elif row['Vol.'][-1] == "M": # volume in millions
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000000
                elif row['Vol.'][-1] == "K": # volume in thousands
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000
                else:
                    print(varname, row['Date'], row['Vol.'])
            rows.append([date, p_close, p_open, p_high, p_low, volume])
        else:
            rows.append([date, p_close, p_open, p_high, p_low])
    df_new = pd.DataFrame(rows)
    if len(rows[0]) == 6:
        df_new.columns = ["Date", varname + "_Close", varname + "_Open", varname + "_High", varname + "_Low", varname + "_Volume"]
    else:
        df_new.columns = ["Date", varname + "_Close", varname + "_Open", varname + "_High", varname + "_Low"]
    
    df_new = df_new.sort_values(by='Date').reset_index(drop=True)
   
    # remove crises
    df_new = df_new[(df_new['Date'] >= datetime(2009, 7, 1)) & (df_new['Date'] <= datetime(2019, 12, 31))]
    
    # add relative difference between open and close to data per day
    df_new = calculate_daily_relative_difference(df_new,  varname + "_Open",  varname + "_Close", varname)
    
    if df.empty:
        df = df_new
    else:
        df = pd.merge(df, df_new, on='Date', how='outer')
    return df

files = {
    # varname: filename
    "S&P500": "Dataset v2/Indices/S&P 500 Historical Data.csv",
    "US30": "Dataset v2/Indices/Dow Jones Industrial Average Historical Data.csv", 
    "US2000": "Dataset v2/Indices/US Small Cap 2000 Historical Data.csv",
    "NASDAQ": "Dataset v2/Indices/NASDAQ Composite Historical Data.csv", 
    "GER30": "Dataset v2/Indices/DAX Historical Data.csv",
    "CAC40": "Dataset v2/Indices/CAC 40 Historical Data.csv",
    "UK100": "Dataset v2/Indices/invUK100 Historical Data.csv",
    "SSE50": "Dataset v2/Indices/Shanghai Composite Historical Data.csv",
    "HS50": "Dataset v2/Indices/Hang Seng Historical Data.csv",
    "NIKKEI225": "Dataset v2/Indices/NIKKEI 225 Historical Data.csv",
}

print(f"Retrieving {len(files)} files")
i = 1
df = pd.DataFrame()
stats_rows = []
for file in files:
    print(f"Retrieving {file} - {i}/{len(files)}")
    df = retrieve_data(df, files[file], file)
    i += 1
df

Retrieving 10 files
Retrieving S&P500 - 1/10
Retrieving US30 - 2/10
Retrieving US2000 - 3/10
Retrieving NASDAQ - 4/10
Retrieving GER30 - 5/10
Retrieving CAC40 - 6/10
Retrieving UK100 - 7/10
Retrieving SSE50 - 8/10
Retrieving HS50 - 9/10
Retrieving NIKKEI225 - 10/10


,Date,S&P500_Close,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Volume,S&P500_relative_change_perc_1,US30_Close,US30_Open,US30_High,...,HS50_High,HS50_Low,HS50_Volume,HS50_relative_change_perc_1,NIKKEI225_Close,NIKKEI225_Open,NIKKEI225_High,NIKKEI225_Low,NIKKEI225_Volume,NIKKEI225_relative_change_perc_1
0,2009-07-01,923.33,920.82,931.92,920.82,0.0,0.002726,8504.06,8447.53,8580.47,...,NaN,NaN,NaN,NaN,9939.93,9889.34,10086.18,9874.00,0.0,0.005116
1,2009-07-02,896.42,921.24,921.24,896.42,0.0,-0.026942,8280.74,8503.00,8503.23,...,18780.96,18053.10,0.0,-0.032102,9876.15,9993.77,9993.77,9869.54,0.0,-0.011769
2,2009-07-06,898.72,894.27,898.72,886.36,0.0,0.004976,8324.87,8279.30,8327.97,...,18258.26,17897.68,0.0,-0.002258,9680.87,9738.49,9771.76,9650.75,0.0,-0.005917
3,2009-07-07,881.03,898.60,898.60,879.93,0.0,-0.019553,8163.60,8324.95,8326.31,...,18159.99,17821.71,0.0,-0.004526,9647.79,9713.62,9734.43,9619.68,0.0,-0.006777
4,2009-07-08,879.56,881.90,886.80,869.32,0.0,-0.002653,8178.41,8157.02,8219.52,...,17819.31,17493.62,0.0,0.003857,9420.75,9548.81,9557.39,9407.98,0.0,-0.013411
5,2009-07-09,882.68,881.28,887.86,878.45,0.0,0.001589,8183.17,8179.01,8228.67,...,17836.95,17509.28,0.0,-0.000170,9291.06,9342.33,9384.24,9291.06,0.0,-0.005488
6,2009-07-10,879.13,880.03,883.57,872.81,0.0,-0.001023,8146.52,8182.49,8183.85,...,17851.22,17645.55,0.0,-0.005148,9287.28,9338.76,9370.15,9265.24,0.0,-0.005513
7,2009-07-13,901.05,879.57,901.05,875.32,0.0,0.024421,8331.68,8146.82,8331.68,...,17612.45,17185.96,0.0,-0.020316,9050.33,9242.13,9343.86,9050.33,0.0,-0.020753
8,2009-07-14,905.84,900.77,905.84,896.50,0.0,0.005629,8359.49,8331.37,8361.23,...,17896.36,17581.43,0.0,0.014409,9261.81,9173.14,9284.38,9166.63,0.0,0.009666
9,2009-07-15,932.68,910.15,933.95,910.15,0.0,0.024754,8616.21,8363.95,8628.23,...,18289.00,18030.10,0.0,0.011976,9269.25,9307.45,9333.64,9250.73,0.0,-0.004104


In [18]:
cols = ["Date"]
for file in files:
    cols.append(file + "_relative_change_perc_1")
    i += 1
cols

['Date',
 'S&P500_relative_change_perc_1',
 'US30_relative_change_perc_1',
 'US2000_relative_change_perc_1',
 'NASDAQ_relative_change_perc_1',
 'GER30_relative_change_perc_1',
 'CAC40_relative_change_perc_1',
 'UK100_relative_change_perc_1',
 'SSE50_relative_change_perc_1',
 'HS50_relative_change_perc_1',
 'NIKKEI225_relative_change_perc_1']

In [23]:
cols = ['Stock Market Index', 'Relative Change per Day']
df_box = pd.DataFrame(columns=cols)

rows1 = []
rows2 = []
for file in files:
    row = df[file + "_relative_change_perc_1"].tolist()
    for i in row:
        new_row = {'Stock Market Index': file, 'Relative Change per Day': i}
        df_box = df_box.append(new_row, ignore_index=True)

In [26]:
fig = px.box(df_box, x='Stock Market Index', y='Relative Change per Day', title="Boxplot of Relative Changes per Day for each Index", )
fig.write_image("Plots/Boxplot Relative Changes.png")
fig.show()